<a href="https://colab.research.google.com/github/Lua-shift/Obra.IA_Alura/blob/main/Obra_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar novos cursos e imersões por lá.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber novidades e anúncios em primeira mão.
*   **Verificar a página de cursos da Alura:** Acesse regularmente a página de cursos da Alura e use a busca para procurar por "Gemini" ou "Inteligência Artificial".

Assim que a Alura divulgar a próxima edição, você ficará sabendo!

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={'tools':[{'google_search': {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições são gratuitas e podem ser feitas até o dia 11 de maio.

A Imersão IA é um curso online com cinco aulas, onde você aprenderá a criar prompts, automatizar tarefas e desenvolver sistemas de agentes autônomos usando o Google Gemini. Ao final do curso, você receberá um certificado da Alura e do Google.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# ----- Agente 1: Buscador de Obra ------ #
##########################################
def agente_buscador_obra(topico, localizacao=None):
    buscador = Agent(
        name="agente_buscador_obra",
        model="gemini-2.0-flash",
        description="Agente que busca informações relevantes para obras no Google.",
        tools=[google_search],
        instruction=f"""
        Você é um assistente de pesquisa para obras de construção civil.

        Sua tarefa é buscar informações atualizadas e úteis no Google sobre o seguinte tópico: {topico}.

        - Se o tópico for sobre preços, procure o valor médio praticado no Brasil ou na cidade informada (se houver).
        - Se for sobre materiais, procure a descrição do item, onde é usado e quanto custa.
        - Se for sobre mão de obra, procure a média de preço por hora ou por dia do profissional mencionado.

        Dê prioridade a resultados de sites confiáveis como lojas de construção, sites de fabricantes e Google Shopping.

        Mostre no máximo 5 resultados úteis, sem repetir.
        Responda de forma simples, clara e objetiva. Inclua preços, nomes e fontes quando possível.

        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}"
    if localizacao:
        entrada_do_agente_buscador += f"\nLocalização do Cliente: {localizacao}"
    informacoes = call_agent(buscador, entrada_do_agente_buscador)
    return informacoes

In [12]:
topico_busca = "preço do saco de cimento 50kg"
local_cliente = "Salvador, Bahia"
resultado_busca = agente_buscador_obra(topico_busca, local_cliente)
print(resultado_busca)

topico_busca = "onde comprar areia ensacada"
local_cliente = "Rio de Janeiro, RJ"
resultado_busca = agente_buscador_obra(topico_busca, local_cliente)
print(resultado_busca)

# Sem localização específica
topico_busca = "melhores práticas para impermeabilização de lajes"
resultado_busca = agente_buscador_obra(topico_busca)
print(resultado_busca)

Em Salvador, Bahia, o preço do saco de cimento de 50kg pode variar. Aqui estão alguns valores encontrados:

*   **OLX:** Anúncios no OLX mostram preços a partir de R$ 32,00.
*   **O Fazendão:** Cimento Poty CP-II 50KG (Votorantim) por R$44,00 à vista. Cimento CP-II F-32 Uso Geral 50KG (Zebu) por R$42,00 à vista.
*   **Outlet Center Líder:** Cimento Montes Claros saco com 50KG por R$37,00.
*   **Santa Lúcia Materiais de Construção:** Cimento Todas as Obras 50 kg Votoran por R$35,00.

É importante verificar a disponibilidade e o frete, pois alguns anúncios são de outras cidades ou podem ter condições especiais.

Os tipos de cimento mais comuns encontrados são o CP II, CP III e CP IV, cada um com indicações específicas para diferentes usos na construção.


Para encontrar areia ensacada no Rio de Janeiro, RJ, aqui estão algumas opções e informações úteis:

1.  **Construtores Center**: Oferece areia ensacada de 20kg a partir de R$7,99. Eles também vendem diversos outros materiais de constru

In [13]:
import json

###################################################
# ----- Agente 2: Planejador de Orçamentos ------ #
###################################################

def agente_planejador_orcamento(tipo_obra, area_m2, localizacao=None):
    planejador = Agent(
        name="agente_planejador_orcamento",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um planejador de orçamentos para construção civil.

        Sua tarefa é indicar os principais materiais e os profissionais necessários para o seguinte serviço:

        Tipo de Obra: {tipo_obra}
        Área: {area_m2} m²
        Localização: {localizacao}

        Retorne a resposta no seguinte formato JSON:

        {{
          "materiais": [
            {{"item": "nome do material", "consulta_preco": "busca no Google para saber o preço"}},
            ...
          ],
          "mao_de_obra": [
            {{"funcao": "nome do profissional", "consulta_preco": "busca no Google para saber a taxa horária"}},
            ...
          ]
        }}

        Inclua na consulta de busca o nome da cidade (se houver). Use termos simples e diretos, como:
        "preço saco de cimento Belo Horizonte", "quanto custa hora de pedreiro em São Paulo".
        """,
        description="Agente que planeja os dados necessários para o orçamento de obras e retorna em JSON.",
        tools=[]
    )

    entrada_do_agente_planejador = f"Tipo de Obra: {tipo_obra}\nÁrea: {area_m2} m²\nLocalização: {localizacao}"

    # Executa o agente
    plano_orcamento_str = call_agent(planejador, entrada_do_agente_planejador)

    try:
        # Limpa delimitadores de markdown (```json ... ```)
        plano_orcamento_limpo = plano_orcamento_str.replace("```json", "").replace("```", "").strip()

        # Carrega como JSON
        plano_orcamento = json.loads(plano_orcamento_limpo)
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar a resposta do Planejador para JSON: {e}")
        print(f"Resposta bruta do Planejador:\n{plano_orcamento_str}")
        return {}  # Retorna um dicionário vazio em caso de erro

    return plano_orcamento

In [14]:
###################################################
# -------- Agente 3: Redator de Orçamentos ------- #
###################################################

def agente_redator_orcamento(tipo_obra, area_m2, plano_orcamento, resultados_busca_preco={}, resultados_busca_mao_de_obra={}):
    redator = Agent(
        name="agente_redator_orcamento",
        model="gemini-2.0-flash",
        instruction="""

Você é um especialista em orçamentos de obras de construção civil.

Use os seguintes critérios para fazer os cálculos:

1. **Para blocos/tijolos**:
   - Considere 20 unidades por metro quadrado de parede.
   - Para calcular a área da parede, use: comprimento x altura (em metros).
   - Exemplo: Uma parede de 5m (comprimento) x 2,6m (altura) = 13 m² → 13 x 20 = 260 blocos.

2. **Para volume de concreto ou outros materiais em volume**:
   - Use a fórmula: comprimento x altura x largura ÷ 1000 (para obter o volume em m³).

3. **Materiais**:
   - Calcule a quantidade com base na área ou volume total, usando consumo médio por m² ou m³, conforme aplicável.
   - Multiplique pela unidade de preço para obter o custo.

4. **Mão de obra**:
   - Use um tempo padrão por m² ou m³ e multiplique pela área ou volume.
   - Multiplique o tempo total pela taxa horária para obter o custo.

Apresente os resultados assim:

Materiais:
- Bloco de concreto: 260 unidades x R$ 1,20 = R$ 312,00

Mão de Obra:
- Pedreiro: 80 horas x R$ 25,00 = R$ 2.000,00

Custo Total Estimado da Obra: R$ 2.312,00

        """,
        description="Agente que calcula e formata orçamentos de obras."
    )

    entrada_do_agente_redator = f"""
Tipo de Obra: {tipo_obra}
Área: {area_m2} m²
Plano de Orçamento: {json.dumps(plano_orcamento, ensure_ascii=False)}
Resultados da Busca de Preços dos Materiais: {json.dumps(resultados_busca_preco, ensure_ascii=False)}
Resultados da Busca de Taxas de Mão de Obra: {json.dumps(resultados_busca_mao_de_obra, ensure_ascii=False)}
"""
    orcamento = call_agent(redator, entrada_do_agente_redator)
    return orcamento

In [15]:
tipo_obra_exemplo = "parede de alvenaria 15x30x20"
area_exemplo = 20
local_exemplo = "Salvador, Bahia"

plano = agente_planejador_orcamento(tipo_obra_exemplo, area_exemplo, local_exemplo)
print("Plano de Orçamento:", plano)

# Aqui o Agente Buscador seria chamado para cada item no plano
# Exemplo simplificado dos resultados da busca (isso viria do agente_buscador_obra):
resultados_preco = {
    "tijolo cerâmico 15x30x20": "R$ 0.80/unidade",
    "cimento": "R$ 30.00/saco de 50kg",
    "areia": "R$ 80.00/m³",
    "cal hidratada": "R$ 15.00/saco de 20kg"
}
resultados_mao_de_obra = {
    "pedreiro": "R$ 35.00/hora",
    "ajudante": "R$ 20.00/hora"
}

orcamento_final = agente_redator_orcamento(tipo_obra_exemplo, area_exemplo, plano, resultados_preco, resultados_mao_de_obra)
print("\nOrçamento Final:\n", orcamento_final)

Plano de Orçamento: {'materiais': [{'item': 'tijolo cerâmico 15x30x20', 'consulta_preco': 'preço tijolo cerâmico 15x30x20 Salvador Bahia'}, {'item': 'cimento', 'consulta_preco': 'preço saco de cimento Salvador Bahia'}, {'item': 'areia', 'consulta_preco': 'preço metro cúbico areia Salvador Bahia'}, {'item': 'cal hidratada', 'consulta_preco': 'preço saco de cal hidratada Salvador Bahia'}, {'item': 'água', 'consulta_preco': 'preço metro cúbico água Salvador Bahia'}, {'item': 'tábua de madeira para caixaria', 'consulta_preco': 'preço tábua de madeira para caixaria Salvador Bahia'}, {'item': 'pregos', 'consulta_preco': 'preço pregos para construção Salvador Bahia'}, {'item': 'arame recozido', 'consulta_preco': 'preço arame recozido Salvador Bahia'}], 'mao_de_obra': [{'funcao': 'pedreiro', 'consulta_preco': 'quanto custa hora de pedreiro em Salvador Bahia'}, {'funcao': 'servente', 'consulta_preco': 'quanto custa hora de servente em Salvador Bahia'}]}

Orçamento Final:
 Com base nos dados for

In [16]:
import json

###################################################
# -------- Agente 4: Revisor de Orçamentos ------- #
###################################################

def agente_revisor_orcamento(tipo_obra, area_m2, orcamento_gerado, resultados_busca_preco={}, resultados_busca_mao_de_obra={}):
    revisor = Agent(
        name="agente_revisor_orcamento",
        model="gemini-2.0-flash",
        instruction=f"""
Você é um revisor de orçamentos para obras de construção civil.

Sua tarefa é verificar se o orçamento gerado para o tipo de obra ({tipo_obra}) com área de {area_m2} m² está coerente e bem calculado.

Regras principais que devem ser seguidas:
- Tijolos/blocos: calcular 20 unidades por metro quadrado de parede.
- Materiais em volume (areia, brita, concreto, etc.): usar fórmula comprimento x altura x largura ÷ 1000.
- Custos: quantidade × preço unitário.
- Mão de obra: tempo estimado × taxa horária.

Verifique os seguintes pontos:
1. Cálculo correto das quantidades com base nas regras acima.
2. Coerência dos preços unitários e das taxas horárias para a região (se disponível).
3. Clareza e organização da apresentação do orçamento.
4. Consistência entre valores totais e subtotais.

Se o orçamento estiver correto, responda:
"O orçamento está revisado e pronto para ser apresentado ao pedreiro."

Se houver problemas, aponte com clareza o que deve ser corrigido.

---

📦 **Dados para revisão:**

Orçamento Gerado:
{json.dumps(orcamento_gerado, indent=2, ensure_ascii=False)}

Resultados da Busca de Preços dos Materiais:
{json.dumps(resultados_busca_preco, indent=2, ensure_ascii=False)}

Resultados da Busca de Taxas de Mão de Obra:
{json.dumps(resultados_busca_mao_de_obra, indent=2, ensure_ascii=False)}
        """,
        description="Agente revisor de orçamentos de obras."
    )

    entrada_do_agente_revisor = f"""
Tipo de Obra: {tipo_obra}
Área: {area_m2} m²
"""
    # Executa o agente
    revisao_orcamento = call_agent(revisor, entrada_do_agente_revisor)
    return revisao_orcamento

In [17]:
from datetime import date
from IPython.display import display, Markdown as md

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🛠️ Iniciando o Sistema de Orçamento de Obras Obra.IA (Sem Simulação) 🛠️")

# --- Obter as informações da obra do usuário ---
tipo_obra = input("❓ Por favor, digite o TIPO DE OBRA que você quer orçar: ")
area_obra = input("📐 Por favor, digite a ÁREA da obra em metros quadrados (m²): ")
localizacao_obra = input("📍 Opcional: Digite a LOCALIZAÇÃO da obra (cidade, estado) para refinar os preços: ")

# Inserir lógica do sistema de agentes Obra.IA (Sem Simulação) ##########################
if not tipo_obra or not area_obra:
    print('Você esqueceu de digitar o tipo da obra e/ou a área!')
else:
    print(f'\n🛠️ Orçamento para {area_obra} m² de {tipo_obra} em andamento...')

    # Agente 2: Planejador de Orçamento (agora gerando consultas de busca)
    plano_orcamento = agente_planejador_orcamento(tipo_obra, area_obra, localizacao_obra)
    print('\n--- Resultado do Agente 2 (Planejador de Orçamento) ---\n')
    display(md(str(plano_orcamento))) # Exibindo o plano (que agora inclui consultas)
    print('--------------------------------------------------------------')

    resultados_preco = {}
    resultados_mao_de_obra = {}

    # Agente 1: Buscador (usando as consultas do Planejador)
    if plano_orcamento and 'materiais' in plano_orcamento:
        print("\n🔍 Buscando preços dos materiais...")
        for material_info in plano_orcamento['materiais']:
            item = material_info.get('item')
            consulta_preco = material_info.get('consulta_preco')
            if item and consulta_preco:
                resultado_busca = agente_buscador_obra(consulta_preco)
                resultados_preco[item] = resultado_busca
                print(f"  - Preço de {item}: {resultado_busca[:50]}...") # Exibe um trecho do resultado

    if plano_orcamento and 'mao_de_obra' in plano_orcamento:
        print("\n🔍 Buscando taxas de mão de obra...")
        for mao_de_obra_info in plano_orcamento['mao_de_obra']:
            funcao = mao_de_obra_info.get('funcao')
            consulta_preco = mao_de_obra_info.get('consulta_preco')
            if funcao and consulta_preco:
                resultado_busca = agente_buscador_obra(consulta_preco)
                resultados_mao_de_obra[funcao] = resultado_busca
                print(f"  - Taxa de {funcao}: {resultado_busca[:50]}...") # Exibe um trecho do resultado

    print('\n--- Resultados da Busca (Agente 1) ---\n')
    print("Preços dos Materiais:", resultados_preco)
    print("Taxas de Mão de Obra:", resultados_mao_de_obra)
    print('--------------------------------------------------------------')

    # Agente 3: Redator de Orçamento (usando os resultados reais da busca)
    orcamento_final = agente_redator_orcamento(
        tipo_obra,
        area_obra,
        plano_orcamento,
        resultados_preco,
        resultados_mao_de_obra
    )
    print('\n--- Resultado do Agente 3 (Redator de Orçamento) ---\n')
    display(md(orcamento_final))
    print('--------------------------------------------------------------')

    # Agente 4: Revisor de Orçamento (usando os resultados reais da busca e o orçamento)
    revisao_orcamento = agente_revisor_orcamento(
        tipo_obra,
        area_obra,
        orcamento_final,
        resultados_preco,
        resultados_mao_de_obra
    )
    print('\n--- Resultado do Agente 4 (Revisor de Orçamento) ---\n')
    display(md(revisao_orcamento))
    print('--------------------------------------------------------------')

print("\n✨ Processo de Orçamento da Obra.IA Concluído! ✨")

🛠️ Iniciando o Sistema de Orçamento de Obras Obra.IA (Sem Simulação) 🛠️
❓ Por favor, digite o TIPO DE OBRA que você quer orçar: construir uma parede
📐 Por favor, digite a ÁREA da obra em metros quadrados (m²): 50
📍 Opcional: Digite a LOCALIZAÇÃO da obra (cidade, estado) para refinar os preços: salvador

🛠️ Orçamento para 50 m² de construir uma parede em andamento...

--- Resultado do Agente 2 (Planejador de Orçamento) ---



{'materiais': [{'item': 'tijolo cerâmico', 'consulta_preco': 'preço tijolo cerâmico Salvador'}, {'item': 'cimento', 'consulta_preco': 'preço saco de cimento Salvador'}, {'item': 'areia', 'consulta_preco': 'preço metro cúbico areia Salvador'}, {'item': 'cal hidratada', 'consulta_preco': 'preço saco de cal hidratada Salvador'}, {'item': 'água', 'consulta_preco': 'preço metro cúbico água Salvador'}, {'item': 'vergalhão (para amarração)', 'consulta_preco': 'preço vergalhão Salvador'}, {'item': 'arame recozido', 'consulta_preco': 'preço arame recozido Salvador'}, {'item': 'ripas de madeira (para caixaria)', 'consulta_preco': 'preço ripa de madeira para caixaria Salvador'}, {'item': 'pregos', 'consulta_preco': 'preço prego para construção Salvador'}], 'mao_de_obra': [{'funcao': 'pedreiro', 'consulta_preco': 'quanto custa hora de pedreiro em Salvador'}, {'funcao': 'servente', 'consulta_preco': 'quanto custa hora de servente em Salvador'}]}

--------------------------------------------------------------

🔍 Buscando preços dos materiais...
  - Preço de tijolo cerâmico: Em Salvador, o preço do tijolo cerâmico pode varia...
  - Preço de cimento: Em Salvador, o preço do saco de cimento de 50kg po...
  - Preço de areia: O preço médio do metro cúbico da areia em maio de ...
  - Preço de cal hidratada: Em Salvador, o preço do saco de cal hidratada de 2...
  - Preço de água: Em Salvador, o preço do metro cúbico da água varia...
  - Preço de vergalhão (para amarração): O preço do vergalhão em Salvador pode variar depen...
  - Preço de arame recozido: O arame recozido é utilizado na construção civil p...
  - Preço de ripas de madeira (para caixaria): Em Salvador, o preço da ripa de madeira para caixa...
  - Preço de pregos: Em Salvador, o preço do prego para construção pode...

🔍 Buscando taxas de mão de obra...
  - Taxa de pedreiro: Em Salvador, o custo da hora de um pedreiro pode v...
  - Taxa de servente: Em Salvador, o custo da 

## Orçamento Detalhado para Construção de Parede (50 m²)

Com base nas informações fornecidas e nos preços de mercado em Salvador, apresento o seguinte orçamento detalhado para a construção de uma parede com área de 50 m²:

### Materiais:

*   **Tijolo Cerâmico:** 1000 unidades x R$ 1,25 = R$ 1.250,00 (Considerando 20 tijolos por m² e preço médio do tijolo cerâmico)
*   **Cimento:** 15 sacos x R$ 42,00 = R$ 630,00 (Estimativa de consumo de 0,3 saco por metro quadrado)
*   **Areia:** 5 m³ x R$ 127,81 = R$ 639,05 (Estimativa de consumo de 0,1 m³ por metro quadrado)
*   **Cal Hidratada:** 3 sacos x R$ 27,27 = R$ 81,81 (Estimativa de consumo de 0,06 saco por metro quadrado)
*   **Água:** 2 m³ x R$ 41,18 = R$ 82,36 (Estimativa de consumo de 0,04 m³ por metro quadrado)
*   **Vergalhão (para amarração):** 2 barras x R$ 41,00 = R$ 82,00 (Estimativa para amarração a cada 25m²)
*   **Arame Recozido:** 2 kg x R$ 14,97 = R$ 29,94 (Estimativa de consumo de 0,04 kg por metro quadrado)
*   **Ripas de Madeira (para caixaria):** 30 metros x R$ 4,66 = R$ 139,80 (Estimativa com base na área da parede)
*   **Pregos:** 1 kg x R$ 15,00 = R$ 15,00 (Estimativa com base no uso geral na caixaria e fixação)

### Mão de Obra:

*   **Pedreiro:** 100 horas x R$ 45,00 = R$ 4.500,00 (Estimativa de 2 horas por metro quadrado)
*   **Servente:** 100 horas x R$ 22,53 = R$ 2.253,00 (Estimativa de 2 horas por metro quadrado)

### Custo Total Estimado da Obra:

R$ 9.702,96

**Observações:**

*   Os preços dos materiais são baseados nos valores de mercado em Salvador, conforme pesquisa realizada em 14 de maio de 2025.
*   Os custos de mão de obra são estimativas e podem variar dependendo da experiência do profissional e das condições do local da obra.
*   Este orçamento não inclui custos adicionais, como transporte de materiais, impostos ou taxas de licenças.
*   Recomenda-se solicitar orçamentos detalhados de fornecedores e profissionais para obter um custo mais preciso da obra.
*   As taxas de consumo de materiais e mão de obra podem variar de acordo com a técnica utilizada e as características do projeto.


--------------------------------------------------------------

--- Resultado do Agente 4 (Revisor de Orçamento) ---



O orçamento está revisado, mas precisa das seguintes correções:

1.  **Tijolo Cerâmico:** A quantidade está correta (1000 unidades para 50 m²), e o preço unitário (R$ 1,25) está dentro da faixa de mercado em Salvador. O subtotal de R$ 1.250,00 está correto.
2.  **Cimento:** O consumo estimado de 0,3 saco por metro quadrado parece alto. Em geral, usa-se cerca de 0,2 saco por m². Sugiro revisar esse valor. Se o valor unitário do saco de cimento está correto (R$ 42,00), o preço está bom.
3.  **Areia:** O consumo estimado de 0,1 m³ por metro quadrado parece alto. Sugiro revisar esse valor. O preço do m³ (R$ 127,81) está de acordo com a pesquisa.
4.  **Cal Hidratada:** O consumo de 0,06 saco por metro quadrado parece um pouco alto. Sugiro revisar esse valor. O preço do saco (R$ 27,27) está bom, de acordo com a pesquisa.
5.  **Água:** O consumo de 0,04 m³ por metro quadrado parece razoável, e o preço do m³ (R$ 41,18) está correto.
6.  **Vergalhão:** A estimativa de 2 barras para 50 m² parece baixa. Vergalhões são usados para dar estrutura à parede, principalmente em amarrações com a estrutura da casa. Sugiro aumentar essa quantidade e detalhar melhor o uso (a cada quantos metros de parede). O preço da barra (R$ 41,00) está coerente.
7.  **Arame Recozido:** O consumo de 0,04 kg por metro quadrado parece razoável, e o preço do kg (R$ 14,97) está bom.
8.  **Ripas de Madeira:** A estimativa de 30 metros parece baixa. É preciso detalhar como essa quantidade foi calculada. O preço do metro (R$ 4,66) está bom.
9.  **Pregos:** A estimativa de 1 kg parece baixa se considerar a caixaria e outras fixações. O preço do kg (R$ 15,00) está correto.
10. **Mão de Obra (Pedreiro e Servente):** A estimativa de 2 horas por metro quadrado para cada um parece alta. O tempo total para construir 50 m² de parede seria de 100 horas para cada um. Sugiro revisar esse tempo. A taxa horária do pedreiro (R$ 45,00) está alta, enquanto a do servente (R$ 22,53) está coerente com a pesquisa.

O valor total do orçamento (R$ 9.702,96) será atualizado automaticamente após as correções nas quantidades e taxas horárias.



--------------------------------------------------------------

✨ Processo de Orçamento da Obra.IA Concluído! ✨
